# Circadian rhythm disruptions

In [ ]:
#| hide
import numpy as np
import matplotlib.pyplot as plt
from circadian.lights import LightSchedule
from circadian.models import Forger99, Jewett99, Hannay19, Hannay19TP

Light exposure is one of the main factors affecting circadian rhythms. Here we use the `circadian` package to explore the effects of light pulses at different times of the day. We will explore this effect using four different circadian models `Forger99`, `Jewett99`, `Hannay19`, and `Hannay19TP` and compare the results among them.

# Entraining models to a regular light schedule

First, we entrain each model to a regular light schedule to have a baseline to compare to. In `circadian` we do this by

In [ ]:
dt = 0.1 # hours
days = 20
time = np.arange(0, 24 * days, dt)
regular_lux = 500
schedule = LightSchedule.Regular(regular_lux, lights_on=8, lights_off=24)
light_input = schedule(time)
model_list = [Forger99(), Jewett99(), Hannay19(), Hannay19TP()]
equilibrium_states = []

for model in model_list:
    time_eq = np.arange(0, 24 * days, dt)
    final_state = model.equilibrate(time_eq, light_input, num_loops=2)
    equilibrium_states.append(final_state)

# Pulse during the day

Next, we can explore how models respond to lights pulse during the bright hours of the day. We can use the `LightSchedule.from_pulse` function to add pulses at different times

In [ ]:
days = 3
time = np.arange(0, 24 * days, dt)
pulse_num = 6
pulse_lux = 1e4
pulse_duration = 1 # hour
start_values = np.linspace(32, 47, pulse_num)

simulation_result = {}

for idx,model in enumerate(model_list):
    simulation_result[str(model)] = {}
    for pulse_start in start_values:
        schedule = LightSchedule.Regular(regular_lux, lights_on=8, lights_off=24)
        schedule += LightSchedule.from_pulse(pulse_lux, pulse_start, pulse_duration)
        light_input = schedule(time)
        trajectory = model(time, equilibrium_states[idx], light_input)
        simulation_result[str(model)][str(pulse_start)] = {
            'light': light_input,
            'trajectory': trajectory
        }

In [ ]:
#| hide
# Generate and save figures
time = time / 24.0

legend_elements = [plt.Rectangle((0, 0), 1, 1, fc='gray', alpha=0.3, ec='black'),
                   plt.Rectangle((0, 0), 1, 1, fc='white', alpha=0.3, ec='black'),
                   plt.Rectangle((0, 0), 1, 1, fc='yellow', alpha=0.3, ec='black'),
                   plt.Line2D([0], [0], color='tab:blue', lw=1),]

for model in model_list:
    figsize_factor = 0.8
    fig, axs = plt.subplots(pulse_num, 1, sharex=True, figsize=(figsize_factor * 15, figsize_factor * 8))

    for idx,start in enumerate(start_values):
        light = simulation_result[str(model)][str(start)]['light']
        trajectory = simulation_result[str(model)][str(start)]['trajectory']

        if str(model) == 'Hannay19':
            y = trajectory.states[:, 0] * np.cos(trajectory.states[:, 1])
        elif str(model) == 'Hannay19TP':
            y = trajectory.states[:, 0] * np.cos(trajectory.states[:, 2])
        elif str(model) in ['Forger99', 'Jewett99']:
            y = trajectory.states[:, 0]

        min_y = 1.5 * min(y)
        max_y = 1.5 * max(y)
        axs[idx].fill_between(time, min_y, max_y, where=light == 0, color='gray', alpha=0.3, edgecolor='None')
        axs[idx].fill_between(time, min_y, max_y, where=light >= pulse_lux-1, color='yellow', alpha=0.3)
        axs[idx].plot(time, y)
        axs[idx].set_ylim(min_y, max_y)
        axs[idx].set_xlim(min(time), max(time)-0.5)
        axs[idx].spines['top'].set_visible(False)
        axs[idx].set_yticks([])
        axs[idx].set_yticklabels([])

    axs[-1].set_xlabel('Days', fontsize=13)
    axs[-1].set_xticks(np.arange(0, 3, 1))
    axs[-1].set_xticklabels(np.arange(0, 3, 1), fontsize=13)
    axs[0].legend(legend_elements, ['Darkness (0 lux)', 'Daylight (500 lux)', 'Pulse (10.000 lux)', 'Core body temperature'], 
                loc='upper left', bbox_to_anchor=(0.05, 1.5), ncols=4, frameon=False, fontsize=11)

    fig.subplots_adjust(hspace=-0.1)
    fig.savefig(f'00_figs/{str(model)}_day_pulses.png', dpi=300, bbox_inches='tight')
    plt.close()

::: {.panel-tabset}

## Forger99

![](00_figs/Forger99_day_pulses.png)

## Jewett99

![](00_figs/Jewett99_day_pulses.png)

## Hannay19

![](00_figs/Hannay19_day_pulses.png)

## Hannay19TP

![](00_figs/Hannay19TP_day_pulses.png)

:::

We see that light pulses during bright hours don't have a major effect on circadian rhythms. However, this is not the case for light pulses during darkness.

# Pulse at night

Using the same code as above but with different pulse start values

In [ ]:
start_values = np.linspace(24, 31, pulse_num)

In [ ]:
#| hide
days = 3
time = np.arange(0, 24 * days, dt)
pulse_num = 6
pulse_lux = 1e4
pulse_duration = 1 # hour

simulation_result = {}

for idx,model in enumerate(model_list):
    simulation_result[str(model)] = {}
    for pulse_start in start_values:
        schedule = LightSchedule.Regular(regular_lux, lights_on=8, lights_off=24)
        schedule += LightSchedule.from_pulse(pulse_lux, pulse_start, pulse_duration)
        light_input = schedule(time)
        trajectory = model(time, equilibrium_states[idx], light_input)
        simulation_result[str(model)][str(pulse_start)] = {
            'light': light_input,
            'trajectory': trajectory
        }

In [ ]:
#| hide
# Generate and save figures for night pulses
time = time / 24.0

legend_elements = [plt.Rectangle((0, 0), 1, 1, fc='gray', alpha=0.3, ec='black'),
                   plt.Rectangle((0, 0), 1, 1, fc='white', alpha=0.3, ec='black'),
                   plt.Rectangle((0, 0), 1, 1, fc='yellow', alpha=0.3, ec='black'),
                   plt.Line2D([0], [0], color='tab:blue', lw=1),]

for model in model_list:
    figsize_factor = 0.8
    fig, axs = plt.subplots(pulse_num, 1, sharex=True, figsize=(figsize_factor * 15, figsize_factor * 8))

    for idx,start in enumerate(start_values):
        light = simulation_result[str(model)][str(start)]['light']
        trajectory = simulation_result[str(model)][str(start)]['trajectory']

        if str(model) == 'Hannay19':
            y = trajectory.states[:, 0] * np.cos(trajectory.states[:, 1])
        elif str(model) == 'Hannay19TP':
            y = trajectory.states[:, 0] * np.cos(trajectory.states[:, 2])
        elif str(model) in ['Forger99', 'Jewett99']:
            y = trajectory.states[:, 0]

        min_y = 1.5 * min(y)
        max_y = 1.5 * max(y)
        axs[idx].fill_between(time, min_y, max_y, where=light == 0, color='gray', alpha=0.3, edgecolor='None')
        axs[idx].fill_between(time, min_y, max_y, where=light >= pulse_lux-1, color='yellow', alpha=0.3)
        axs[idx].plot(time, y)
        axs[idx].set_ylim(min_y, max_y)
        axs[idx].set_xlim(min(time), max(time)-0.5)
        axs[idx].spines['top'].set_visible(False)
        axs[idx].set_yticks([])
        axs[idx].set_yticklabels([])

    axs[-1].set_xlabel('Days', fontsize=13)
    axs[-1].set_xticks(np.arange(0, 3, 1))
    axs[-1].set_xticklabels(np.arange(0, 3, 1), fontsize=13)
    axs[0].legend(legend_elements, ['Darkness (0 lux)', 'Daylight (500 lux)', 'Pulse (10.000 lux)', 'Core body temperature'], 
                loc='upper left', bbox_to_anchor=(0.05, 1.5), ncols=4, frameon=False, fontsize=11)

    fig.subplots_adjust(hspace=-0.1)
    fig.savefig(f'00_figs/{str(model)}_night_pulses.png', dpi=300, bbox_inches='tight')
    plt.close()

gives us the following 

::: {.panel-tabset}

## Forger99

![](00_figs/Forger99_night_pulses.png)

## Jewett99

![](00_figs/Jewett99_night_pulses.png)

## Hannay19

![](00_figs/Hannay19_night_pulses.png)

## Hannay19TP

![](00_figs/Hannay19TP_night_pulses.png)

:::

We see that models are more sensitive to pulses during the dark hours of the day--the sinusoidal signal changes abruptly when the pulse is applied. This reflects an important property of circadian rhythms: their sensitivity to light is dependent on the current phase of the clock. We can calculate how much the phase of the clock changes after a pulse by constructing a phase response curve (PRC).

# Building phase response curves

WIP